In [14]:
import os
import sys
import numpy as np
from numpy import genfromtxt
import pickle

#hyperParameters
C = 1.0
T = 10000
k = 100



def read_data(input_file):
	X = genfromtxt(input_file,delimiter = ',') # list of training example vectors
	Y = X[:,len(X[0])-1]
	Y = Y[np.newaxis]
	Y = np.transpose(Y)
	X = np.delete(X,len(X[0])-1,1)
	X = np.array(X,dtype="float64")
	X = X/255.0
# 	x0 = np.ones((len(X),1))
# 	X = np.hstack((x0,X))
	return X,Y

In [15]:
X,Y = read_data('data/train.csv')

In [16]:
test_X,test_Y = read_data('data/test.csv')

In [17]:
def pegasos(X_part,Y_part):
	global k
	m = len(X_part)
	k = min(m,k)
	w = np.zeros((len(X_part[0]),1))
	b = 0
	for t in range(T):
		k_array = np.random.choice(m,k)
		step_neta = C * (1.0/(t+1))
		update_amount = np.zeros((len(X_part[0]),1))
		update_b = 0
		for i in range(k):
			X_i = X_part[k_array[i]]
			X_w = np.matmul(X_i,w)
			y_X_w = Y_part[k_array[i]] * X_w
			if y_X_w + Y_part[k_array[i]] * b < 1:
				X_i = X_i[np.newaxis]
				update_amount += Y_part[k_array[i]] * np.transpose(X_i)
				update_b += Y_part[k_array[i]]                
		w = (1 - step_neta*(1/C)) * w + (step_neta/k) * update_amount
		b = b + (step_neta/k) * update_b
	new_w = np.zeros((len(X_part[0])+1,1))
	for i in range(len(X_part[0])):
		new_w[i] = w[i]
	new_w[len(X_part[0])][0] = b
	return new_w

In [18]:
def learn_parameters():
	m = len(X)
	labels_array = np.unique(Y)
	labels = len(labels_array)
	classifier = np.zeros((labels,labels,(len(X[0])+1),1))
	for i in range(labels):
		for j in range(i+1,labels):
			X_part = []
			Y_part = []
			for l in range(m):
				if Y[l][0]==i or Y[l][0]==j:
					X_part.append(X[l])
					if Y[l][0]==i:
						Y_part.append(1)
					else:
						Y_part.append(-1)
			w = pegasos(X_part,Y_part)
			classifier[i][j] = w
			classifier[j][i] = w
	return classifier

In [19]:
classifier = learn_parameters()
with open('./pickles/svm1.pkl', 'wb') as f:
    pickle.dump(classifier,f)

In [33]:
def predict_label(test_X):
	test_X = np.append(test_X,[1])
	labels_array = np.unique(Y)
	labels = len(labels_array)
	wins = np.zeros((labels))
	for i in range(labels):
		for j in range(i+1,labels):
			dist = np.matmul(test_X,classifier[i][j])
			if dist > 0:
				wins[i] += 1
			elif dist < 0:
				wins[j] += 1
	label = -1
	maxx = -1
	for i in range(labels):
		if wins[i] >= maxx:
			maxx = wins[i]
			label = i
	return label

In [34]:
def accuracy(test_data,test_labels):
	count = 0
	for i in range(len(test_data)):
		predicted_label = predict_label(test_data[i])
		expected_label = test_labels[i][0]
		if predicted_label == expected_label:
			count += 1
	return (1.0*count/len(test_data))

In [35]:
train_accuracy = accuracy(X,Y)

In [36]:
print train_accuracy

0.8867


In [37]:
test_accuracy = accuracy(test_X,test_Y)

In [38]:
print test_accuracy

0.901


In [13]:
# confuse = np.zeros((10,10),dtype='int64')
# f = open('guassian_10_output.txt','r')
# for i in range(len(test_Y)):
#     predicted = int(f.readline())
#     actual = int(test_Y[i][0])
#     if not actual ==predicted:
#         print actual,predicted
#     confuse[actual][predicted] +=1
# print confuse

4 0
7 4
9 8
9 8
4 2
2 7
0 5
5 3
3 7
2 8
6 0
9 8
3 5
8 2
8 3
8 2
8 3
2 8
1 8
3 9
2 8
7 3
7 9
4 9
0 6
5 8
4 9
4 9
7 2
8 3
8 3
9 7
1 3
3 5
8 9
5 4
1 2
6 0
6 5
7 9
6 8
9 8
4 6
6 1
6 8
7 2
9 4
4 9
9 0
7 1
7 2
5 7
8 3
7 9
5 6
5 3
7 9
7 1
7 9
1 5
8 7
4 6
9 3
9 7
9 3
7 9
3 7
9 3
8 0
7 3
5 3
7 2
2 8
6 4
8 5
1 4
9 4
7 2
9 3
7 2
5 3
4 8
2 7
7 4
2 0
3 7
6 0
9 2
4 9
6 1
1 2
0 5
9 1
1 6
8 0
9 0
2 8
7 2
9 1
9 8
6 4
2 4
5 8
7 2
2 8
9 5
6 1
7 4
4 9
9 4
8 0
3 2
9 5
3 5
6 5
9 7
1 2
4 6
5 9
8 3
7 1
6 0
2 3
7 9
9 7
2 6
6 0
3 7
9 1
6 4
3 8
5 0
8 5
8 5
9 3
7 0
8 3
8 0
2 3
7 2
5 8
4 6
2 8
5 8
7 8
2 3
6 5
5 6
5 3
1 3
9 3
4 6
7 1
9 4
0 2
8 3
9 2
8 2
9 5
2 7
7 9
1 9
9 7
2 8
2 7
9 0
5 8
3 7
5 6
2 8
8 7
7 8
9 1
4 2
8 9
8 3
7 2
6 1
3 5
9 4
8 3
9 4
7 2
8 6
0 8
2 8
2 3
8 4
3 2
3 2
1 6
7 9
1 5
3 7
7 9
7 2
4 2
4 9
5 3
3 8
3 8
2 0
3 8
9 3
9 8
9 8
9 8
9 8
9 3
9 8
9 8
9 2
5 2
5 2
3 2
9 0
8 9
9 4
9 7
0 7
5 6
8 7
9 5
8 5
0 5
0 6
7 2
4 8
5 2
3 2
3 2
3 2
2 4
3 2
1 8
2 8
7 2
0 6
9 7
8 6
8 6
4 9
7 2
2 8
7 2
7 2
7 2
7 2
7 2
8 5


In [11]:
# preci = np.zeros((10),dtype='float64')
# for i in range(10):
#     summ = 0
#     for j in range(10):
#         summ += confuse[i][j]
#     preci[i] = confuse[i][i] / (1.0 * summ)


In [12]:
# print preci

[ 0.98877551  0.98854626  0.96899225  0.97524752  0.9796334   0.97085202
  0.98121086  0.95914397  0.96714579  0.94846383]
